<a href="https://colab.research.google.com/github/Abudhagir/EVA8/blob/main/EVA8_S3_Assignment_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 8, 3, bias=False)  # Input - 28, Output - 26, RF  - 3
        self.bn1   = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, bias=False) # Input - 26, Output - 24, RF  - 5
        self.bn2   = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)               # Input - 24, Output - 12, RF  - 10
        self.conv3 = nn.Conv2d(16, 16, 3, bias=False) # Input - 12, Output - 10, RF  - 12
        self.bn3   = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 32, 3, bias=False) # Input - 10, Output - 8, RF  - 14
        self.bn4   = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)               # Input - 8, Output - 4, RF  - 28
        self.conv5 = nn.Conv2d(32, 10, 4, bias=False) # Input - 4, Output - 2, RF  - 30        
        self.dropout = nn.Dropout2d(0.1)
    def forward(self, x):
        x = self.pool1(self.dropout(F.relu(self.bn2(self.conv2(self.dropout(F.relu(self.bn1(self.conv1(x)))))))))
        x = self.pool2(self.dropout(F.relu(self.bn4(self.conv4(self.dropout(F.relu(self.bn3(self.conv3(x)))))))))
        x = self.conv5(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [4]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
       BatchNorm2d-2            [-1, 8, 26, 26]              16
         Dropout2d-3            [-1, 8, 26, 26]               0
            Conv2d-4           [-1, 16, 24, 24]           1,152
       BatchNorm2d-5           [-1, 16, 24, 24]              32
         Dropout2d-6           [-1, 16, 24, 24]               0
         MaxPool2d-7           [-1, 16, 12, 12]               0
            Conv2d-8           [-1, 16, 10, 10]           2,304
       BatchNorm2d-9           [-1, 16, 10, 10]              32
        Dropout2d-10           [-1, 16, 10, 10]               0
           Conv2d-11             [-1, 32, 8, 8]           4,608
      BatchNorm2d-12             [-1, 32, 8, 8]              64
        Dropout2d-13             [-1, 32, 8, 8]               0
        MaxPool2d-14             [-1, 3

<ipython-input-3-51bf8853fe99>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [7]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-3-51bf8853fe99>:22: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.07747787237167358 batch_id=468: 100%|██████████| 469/469 [01:01<00:00,  7.57it/s]



Test set: Average loss: 0.0665, Accuracy: 9790/10000 (98%)



loss=0.0434943288564682 batch_id=468: 100%|██████████| 469/469 [00:59<00:00,  7.85it/s]



Test set: Average loss: 0.0475, Accuracy: 9840/10000 (98%)



loss=0.056884367018938065 batch_id=468: 100%|██████████| 469/469 [01:01<00:00,  7.68it/s]


...


Test set: Average loss: 0.0214, Accuracy: 9927/10000 (99%)



loss=0.05325852334499359 batch_id=468: 100%|██████████| 469/469 [00:59<00:00,  7.93it/s]



Test set: Average loss: 0.0234, Accuracy: 9922/10000 (99%)



loss=0.03662116080522537 batch_id=468: 100%|██████████| 469/469 [00:57<00:00,  8.17it/s]



Test set: Average loss: 0.0213, Accuracy: 9932/10000 (99%)

